In [ ]:
import sys

# Install brave-search package if not already installed
# Use the magic command only at the top level, not inside a code block or script
%pip install brave-search

In [ ]:
# just use config.py. For this prototype's purpose, we'll hardcode this here.
import json
import os

config_path = r"D:\Ennchan\OneDrive\Documents\sandbox\ennchan-langchain-rag\config.json"

try:
    # Load the configuration
    with open(config_path, "r") as f:
        config = json.load(f)
    
    print(f"Config loaded successfully with {len(config)} settings")

except FileNotFoundError:
    print("File not found")


Config loaded successfully with 11 settings


In [ ]:
# InvokeSearch
from brave import Brave
from pprint import pprint as pr

os.environ["BRAVE_API_KEY"] = config["BRAVE_API_KEY"]
brave = Brave()

query = "cobalt mining"
num_results = 10

search_results = brave.search(q=query, raw=True)
output = search_results["web"]["results"]

In [9]:
# SearchResultClass
search_results = [{
    "title": result["title"],
    "url": result["url"],
    "description": result["description"],
} for result in output]


In [11]:
# Plaintext for the webloader
search_result = search_results[0]

import requests
from bs4 import BeautifulSoup

url = search_result["url"]

response = requests.get(url)
response.raise_for_status()  # Raises an error if the request failed

soup = BeautifulSoup(response.text, "html.parser")
plaintext = soup.get_text(separator="\n", strip=True)

print(plaintext)

Cobalt Mining: The Dark Side of the Energy Transition | Earth.Org
Membership
ABOUT
Search
Sign Up
Membership
Donate
Email
*
Phone
This field is for validation purposes and should be left unchanged.
home_icon-01_outline
star
Climate Change
Energy
Pollution
Policy & Economics
Oceans
Biodiversity
Conservation
Solutions
Guides
ABOUT
SUPPORT
Earth.Org Newsletters
Get focused newsletters especially designed to be concise and easy to digest
Subscription Options
*
ESSENTIAL BRIEFING
3 times weekly
TOP STORY ROUNDUP
Once a week
MONTHLY OVERVIEW
Once a month
Enter your email
*
Comments
This field is for validation purposes and should be left unchanged.
Earth.Org
PAST · PRESENT · FUTURE
Environmental News, Data Analysis, Research & Policy Solutions.
Read Our Mission Statement
About EO
About Us
Impact & Reach
Our Planet’s Trajectory
Our Team
Meet The Team
Our Partners
Global Ambassadors
Global Photographers
NGO Partners
Energy
Cobalt Mining: The Dark Side of the Renewable Energy Transition
by
Aphr